# 使用email文件夹下spam文件夹(垃圾邮件)和ham文件夹（正常邮件）的共50个邮件，采用交叉验证的方式，随机选取10个文件作为测试数据，其他作为训练数据，采用朴素贝叶斯进行垃圾邮件的分类。计算多次迭代后的平均错误率。（本题要求不使用sklearn库）

In [13]:
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

from param.parameterized import classlist
from sklearn.naive_bayes import MultinomialNB

def createVocabList(dataSet):
    vocabSet=set([])
    for document in dataSet:
        vocabSet=vocabSet|set(document)
    return list(vocabSet)

def setOfWrods2Vec(vocabList,inputSet):
    returnVec=[0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)]=1
        else:
            print('the word: %s is not in my Vocabulary!'% word)
    return returnVec

def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory) / float(numTrainDocs)
    p0Num = np.zeros(numWords)
    p1Num = np.zeros(numWords)
    p0Denom = 0.0
    p1Denom = 0.0
    for i in range(numTrainDocs):
        if trainCategory[i]==1:
            p1Num+=trainMatrix[i]
            p1Denom+=sum(trainMatrix[i])
        else:
            p0Num+=trainMatrix[i]
            p1Denom+=sum(trainMatrix[i])
    p1Vect=p1Num/p1Denom
    p0Vect=p0Num/p0Denom
    return p0Vect,p1Vect,pAbusive

def classifyNB(vec2Classify,p0Vec,p1Vec,pClass1):
    p1=sum(vec2Classify*p1Vec)+np.log(pClass1)
    p0=sum(vec2Classify*p0Vec)+np.log(1-pClass1)
    if p1>p0:
        return 1
    else:
        return 0

def bagOfWords2VecMN(vocabList,inputSet):
    returnVec=[0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)]+=1
    return returnVec

def textParse(bigString):
    listOfTokens=re.split(r'\W*',bigString)
    return [tok.lower() for tok in listOfTokens if len(tok)>2]

def spamTest():
    docList=[]
    classList=[]
    fullText=[]
    for i in range(1,26):
        wordList=textParse(open('C:/Users/Admin/Desktop/WHU study/programming/python/MachineLearning/exp3/email/spam/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList=textParse(open('C:/Users/Admin/Desktop/WHU study/programming/python/MachineLearning/exp3/email/ham/%d.txt' % i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList=createVocabList(docList)
    trainingSet=range(50)
    testSet=[]
    for i in range(10):
        randIndex=int(random.uniform(0,len(trainingSet)))
        testSet.append(list(trainingSet)[randIndex])
        del(list(trainingSet)[randIndex])
    trainMat=[]
    trainClasses=[]
    for docIndex in trainingSet:
        trainMat.append(setOfWrods2Vec(vocabList,docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam=trainNB0(np.array(trainMat),np.array(trainClasses))
    errorCount=0
    for docIndex in testSet:
        wordVector=setOfWrods2Vec(vocabList,docList[docIndex])
        if classifyNB(np.array(wordVector),p0V,p1V,pSpam)!=classList[docIndex]:
            errorCount+=1
    print('the error rate is: ',float(errorCount)/len(testSet))
    
spamTest()
# error:错误率太大

the error rate is:  0.4


# 采用sklearn中的MultinomialNB朴素贝叶斯算法对sklearn的datasets中自带的digits数据集进行分类，测试分类效果。


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report,confusion_matrix

digits=datasets.load_digits()
X=digits.data
y=digits.target
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=33)
mm=MinMaxScaler()
X_train=mm.fit_transform(X_train)
X_test=mm.transform(X_test)
mnb=MultinomialNB()
mnb.fit(X_train,y_train)
y_predict=mnb.predict(X_test)
print('The accuracy of Naive Bayes Classifier is',mnb.score(X_test,y_test))
print(classification_report(y_test,y_predict))
print(confusion_matrix(y_test,y_predict))


The accuracy of Naive Bayes Classifier is 0.8844444444444445
              precision    recall  f1-score   support

           0       0.97      1.00      0.99        35
           1       0.81      0.85      0.83        54
           2       0.93      0.93      0.93        44
           3       0.98      0.87      0.92        46
           4       0.91      0.91      0.91        35
           5       0.95      0.75      0.84        48
           6       0.98      0.98      0.98        51
           7       0.83      0.97      0.89        35
           8       0.85      0.81      0.83        58
           9       0.71      0.84      0.77        44

    accuracy                           0.88       450
   macro avg       0.89      0.89      0.89       450
weighted avg       0.89      0.88      0.88       450

[[35  0  0  0  0  0  0  0  0  0]
 [ 0 46  2  0  0  0  0  0  1  5]
 [ 1  2 41  0  0  0  0  0  0  0]
 [ 0  0  1 40  0  1  0  1  2  1]
 [ 0  0  0  0 32  0  0  1  2  0]
 [ 0  0  0  0  

# 对sklearn自带的breast_cancer数据集，该数据集包含569个病例的数据样本，每个样本具有30个特征值，样本共分为2类:恶性和良性。采用朴素贝叶斯分类器、交叉验证法进行测试，计算训练集准确率（score）、测试集准确率（score）。

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import classification_report,confusion_matrix

cancer=datasets.load_breast_cancer()
X=cancer.data
y=cancer.target
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=33)

gnb=GaussianNB()
mnb=MultinomialNB()
bnb=BernoulliNB()
cnb=ComplementNB()

sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

mm=MinMaxScaler()
X_train2=mm.fit_transform(X_train)
X_test2=mm.transform(X_test)

gnb.fit(X_train,y_train)
mnb.fit(X_train2,y_train)
bnb.fit(X_train,y_train)
cnb.fit(X_train2,y_train)

y_predict=gnb.predict(X_test)
print('The accuracy of Naive Bayes Classifier is',gnb.score(X_test,y_test))
print('The accuracy of Naive Bayes Classifier is',gnb.score(X_train,y_train))
scores=cross_val_score(gnb,X,y,cv=5)
print('The cross_val_score of Naive Bayes Classifier is',np.mean(scores))
print(classification_report(y_test,y_predict))
print(confusion_matrix(y_test,y_predict))

y_predict=mnb.predict(X_test2)
print('The accuracy of Naive Bayes Classifier is',mnb.score(X_test2,y_test))
print('The accuracy of Naive Bayes Classifier is',mnb.score(X_train2,y_train))
scores=cross_val_score(mnb,X,y,cv=5)
print('The cross_val_score of Naive Bayes Classifier is',np.mean(scores))
print(classification_report(y_test,y_predict))
print(confusion_matrix(y_test,y_predict))

y_predict=bnb.predict(X_test)
print('The accuracy of Naive Bayes Classifier is',bnb.score(X_test,y_test))
print('The accuracy of Naive Bayes Classifier is',bnb.score(X_train,y_train))
scores=cross_val_score(bnb,X,y,cv=5)
print('The cross_val_score of Naive Bayes Classifier is',np.mean(scores))
print(classification_report(y_test,y_predict))
print(confusion_matrix(y_test,y_predict))

y_predict=cnb.predict(X_test2)
print('The accuracy of Naive Bayes Classifier is',cnb.score(X_test2,y_test))
print('The accuracy of Naive Bayes Classifier is',cnb.score(X_train2,y_train))
scores=cross_val_score(cnb,X,y,cv=5)
print('The cross_val_score of Naive Bayes Classifier is',np.mean(scores))
print(classification_report(y_test,y_predict))
print(confusion_matrix(y_test,y_predict))

The accuracy of Naive Bayes Classifier is 0.951048951048951
The accuracy of Naive Bayes Classifier is 0.9460093896713615
The cross_val_score of Naive Bayes Classifier is 0.9385188635305075
              precision    recall  f1-score   support

           0       0.98      0.89      0.93        54
           1       0.94      0.99      0.96        89

    accuracy                           0.95       143
   macro avg       0.96      0.94      0.95       143
weighted avg       0.95      0.95      0.95       143

[[48  6]
 [ 1 88]]
The accuracy of Naive Bayes Classifier is 0.8321678321678322
The accuracy of Naive Bayes Classifier is 0.8403755868544601
The cross_val_score of Naive Bayes Classifier is 0.8963204471355379
              precision    recall  f1-score   support

           0       0.97      0.57      0.72        54
           1       0.79      0.99      0.88        89

    accuracy                           0.83       143
   macro avg       0.88      0.78      0.80       143
wei